# Stim-Based Memory Experiment for d=3 Surface Code

This notebook uses Stim to perform a noisy memory experiment to evaluate the robustness of different CNOT orderings for the d=3 surface code. The primary goal is to see if the permutations identified as 'good' by the PyZX analysis also show a lower logical error rate in a simulation with depolarizing noise.

The experiment proceeds as follows:
1. For each of the 24 possible CNOT permutations for the weight-4 stabilizers:
2. A multi-round Stim circuit is constructed with a specified level of 2-qubit depolarizing noise.
3. The circuit is simulated for a number of shots to measure the logical Z error rate.
4. This process is repeated multiple times to gather statistics (mean, standard deviation, and standard error).
5. The results are then sorted to identify the most robust CNOT orderings under these noise conditions.

In [ ]:
# === 1. Imports and Setup ===
import matplotlib.pyplot as plt
from zx_error_prop.stim_interface import run_stim_memory_experiment

# --- Experiment Parameters ---
# Number of syndrome measurement rounds
ROUNDS = 6

# 2-qubit depolarizing error probability after each CNOT
P2_NOISE = 0.00002

# Number of times the circuit is sampled per repeat
SHOTS_PER_REPEAT = 5000

# Number of times to repeat the whole simulation for statistical significance
REPEATS = 10

In [ ]:
# === 2. Run Experiment and Display Results ===

total_shots_per_perm = SHOTS_PER_REPEAT * REPEATS

results = run_stim_memory_experiment(
    rounds=ROUNDS,
    p2=P2_NOISE,
    shots=SHOTS_PER_REPEAT,
    repeats=REPEATS
)

print(f"\nAveraged over {REPEATS} repeats x {SHOTS_PER_REPEAT} shots each (total shots per perm = {total_shots_per_perm})")
print("Permutation -> Mean Logical Z Error Rate (Std. Err.)")
print("-" * 50)

sorted_results = sorted(results.items(), key=lambda item: item[1]['mean'])

for perm, stats in sorted_results:
    mean, stderr = stats['mean'], stats['std_err']
    print(f"{str(perm):<15} -> {mean:.5f} (± {stderr:.5f})")

# Find and print the best permutation
best_perm, best_stats = sorted_results[0]
print("\n--- Best Permutation ---")
print(f"Permutation: {best_perm}")
print(f"Mean Failure Rate: {best_stats['mean']:.5f}")